### This notebook shows the result for Energy_resolution in case of fully recovered shower.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import TheilSenRegressor, LinearRegression, HuberRegressor, RANSACRegressor
from random import seed
from random import randrange

In [20]:
df = pd.read_csv('data_for_clustering.csv', index_col=[0])

/root/miniconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [21]:
X = []
E_true = []
for i in range(41):
    df_ = df[df.Brick_id == i]
    X += df_['ele_P'].value_counts().to_list()
    E_true +=  [i for i in df_['ele_P'].unique()]

In [22]:
print(len(X), len(E_true))

7999 7999


In [23]:
seed(1)

# Split a dataset into k folds
def cross_validation_split(dataset, folds=5):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / folds)
    for i in range(folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split



df_ = df[df.Brick_id == 0.0]
X = np.array(df_['ele_P'].value_counts().to_numpy())
X = X.reshape((-1, 1))
print(X.shape)
E_true =  df_['ele_P'].unique()


folds = cross_validation_split(X, 5)
y = cross_validation_split(E_true, 5)

E_pred = []
for i in range(len(folds)):
    folds_ = folds.copy()
    y_ = y.copy()
        
    r = HuberRegressor()
    X = folds_.pop(i)
    Y = y_.pop(i)
        
    folds_new = np.array([item for sublist in folds_ for item in sublist])
    y_new = np.array([item for sublist in y_ for item in sublist])

    r.fit(folds_new, y_new, sample_weight=1/(y_new)**6)
        
    Y_pred = r.predict(X)      
    print(np.std((Y - Y_pred) / Y))
        
    E_pred.append(Y_pred)
    
             
scale_mm = 10000
Energy_resolution = np.std((E_true - np.mean(E_pred)) / E_true)

(200, 1)
0.2426376482513215
0.2948957477932015
0.26171988931916074
0.22992464429284393
0.22237412629813474


In [24]:
Energy_resolution

0.25456612791152866